In [3]:
import numpy as np, torch, matplotlib.pyplot as plt, PIL
from IPython.display import Image

plt.style.use(['science', 'notebook', 'grid', 'dark_background'])

# An Initial Problem:

Suppose we have a dataset $\vec p=[1,3,5,2...]$ of length $N$. We wanna know the corresponding matching dataset $\vec q=[q_1,q_2,q_3..]$ is such that this Loss Function $$H(\vec p,\vec q)=-\sum_{i=1}^Np_i\,\ln(q_i)$$  is minimized with the constraint $\sum_i p_i=\sum_i q_i$.

In [4]:
p = np.array([5, 1, 4, 6, 2, 4])
q1 = p

q2 = np.array([3, 7, 1, 4, 1, 6])
q3 = np.array([2, 5, 7, 2, 1, 5])

-sum(p * np.log(q2))

-22.923775636027425

Turns out that the set of numbers that minimizes $H$ is exactly $q=p$. We can prove this.

We wanna maximize a function with respect to a constriant, so this is a Lagrange Multiplier problem. We find the stationary points of $$f(\vec q,\lambda)\equiv -\sum_{i=1}^Np_i\,\ln(q_i)=-\lambda\Bigg(\sum_i p_i-\sum_i q_i\Bigg)$$

We set the first derivative with respect to each of $q_i$ equal to zero, and also that respect to $\lambda$: $$\frac{\partial f}{\partial q_i}=-\frac{p_i}{q_i}+\lambda \to 0$$ and $$\frac{\partial f}{\partial \lambda}=\Bigg(\sum_i p_i-\sum_i q_i\Bigg)\to 0$$

From this we get
- $p_i=\lambda q_i \implies \sum_ip_i=\lambda \sum_i q_i$
- $\sum_ip_i=\sum_iq_i$

These both can only be true if $\lambda=1$ so therefore we must have $q_i=p_i$ and thus $\vec q=\vec p$.

This also works for functions as well. In this case, rather than having $\vec q$ and $\vec p$ where $\sum_i q_i=\sum_i q_i$, we'd have $\int_{\mathbb R}p(x)\,dx=\int_{\mathbb R}q(x)\,dx$. The function $q(x)$ that minimizes $$\int_{\mathbb R}p(x)\,\ln(q(x))\,dx$$ is precisely $q(x)=p(x)$. To prove this, one needs calculus of variation. This is why this function represents 'cross' entropy as it corresponds to a way to measure the difference of a measured function $q$ with respect to a known function $p$.

# Cross-Entropy Loss:

In Classification problems, an input (usually an image) is taken in, which we'll call $x$. This image may for example, contain 1 of 5 different objects. We'll call the true likelihood of an image $x$ belonging to a class $i$ as $p_i$. The goal of a Classifier is to create a function $f$ such that $$f(x)=\vec q$$ where $\vec q$ is as close to $\vec p$ as possible.

- $\vec p$ and $\vec q$ are Probability Mass Functions, and each element of the vector represents a different class. It follows that $\sum_i p_i=\sum_iq_i=1$, the constraint that we had in the Lagrange Multiplier problem above.
- Typically, we know what class $\tilde C$ an image $x$ belongs to. In this case, its typically the case that $p_{\tilde C}=1$ for the class $i=\tilde C$, that we know it is, and all the other $p_i=0$.

To minimize the difference between $\vec p$ and $\vec q$ we can minimize the Loss Function $$H(p,q)=-\sum_ip_i\,\ln(q_i)$$ Since the minima of this function occurs exactly when $q_i=p_i \,\forall\, i$, we know which class an image belongs to: $$H(p,q)=-\ln(q_i)$$

In [6]:
p = np.zeros(10, dtype = int); p[4] = 1
q = np.random.rand(10)

q = q / sum(q)
p, q

(array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0]),
 array([0.08389358, 0.13963495, 0.15329469, 0.16223148, 0.05212778,
        0.04602886, 0.14695855, 0.00457714, 0.19063502, 0.02061793]))

In [7]:
H = -np.log(q[p > 0])
H

array([2.95405732])

At this point, we'll see what happens when we make the probability higher:

In [8]:
q[4] = 20
q = q / sum(q)

H = -np.log(q[p > 0])
H

array([0.0463048])

# For Multiple Images:

In this case, consider we compute this Loss over $N$ images so that the $N$ th one is $x_N$. Suppose also that we know exactly which class the image belongs to. We can thus express the true class of the $n$ th image as $\tilde C(n)$. We'll express the probability of an image $n$ belonging to a class $\tilde C$ as $q_n(\tilde C)$.

Thus the predicted probability of the image $x_n$ belonging to its true class $\tilde C(n)$ is $q_n(\tilde C(n))$ and we sum it together: $$L(p,q)\equiv\sum_{n=1}^NH(p_n,q_n)=-\sum_{n=1}^N\ln(q_n(\tilde C(n)))$$

In [9]:
p = np.zeros((4, 10), dtype = int)

p[0][4] = 1
p[1][2] = 1
p[2][8] = 1
p[3][6] = 1

p

array([[0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]])

In [11]:
q = np.random.rand(40).reshape(4, 10)
q = q / np.expand_dims(np.sum(q, axis = 1), axis = 1)

q

array([[0.17182178, 0.17354272, 0.00264357, 0.11538858, 0.0952921 ,
        0.03923513, 0.13330965, 0.02834258, 0.15330713, 0.08711677],
       [0.15093522, 0.11082929, 0.00041901, 0.18078421, 0.13535436,
        0.0160621 , 0.06940709, 0.02145562, 0.15551819, 0.15923491],
       [0.10116303, 0.00282392, 0.01644199, 0.02477249, 0.14041257,
        0.08760147, 0.13662187, 0.20654316, 0.09503555, 0.18858395],
       [0.09795096, 0.17183749, 0.05300985, 0.11756661, 0.18161321,
        0.09791736, 0.07609876, 0.04298613, 0.00126308, 0.15975655]])

In [12]:
H_values = -np.log(q[p > 0])
L = sum(H_values)

L

15.05766254209766

$\vec q$ should be related to a Probability Density Function where

1. Bounded between 0 and 1,
2. The closer we get to 0, the less likely we're confident that image $n$ is in class $C$,
3. The closer we get to 1, the more likely we're confident that image $n$ is in class $C$, and
4. $\sum_{i=0}^Cq_C=1$ for each image.

Suppose a Neural Network outputs $f(x_n)=\hat y_n$ where $\hat y_n$ is a vector with the same length as the number of classes, but it's not normalized like $\vec q$ should be. We can enforce the last condition by normalizing the following way: $$q_n(C)=\frac{\exp(\hat y_n(C))}{\sum_{C'=0}^C\exp(y_n(C'))}$$ So we can write our Loss $$L(\hat y_n)=-\sum_{n=0}^N\ln(q_n(\tilde C(n)))=-\sum_{n=0}^N\ln\Bigg[\frac{\exp(\hat y_n(\tilde C(n)))}{\sum_{C'=0}^C\exp(\hat y_n(C'))}\Bigg]$$ 

For a given $n$, $\hat y_n(C)$ is the Network Output for the image $x_n$. It's a vector of length 10 (the network outputs 10 numbers per image).

In [14]:
yhat = 20 * np.random.rand(40).reshape(4, 10) ** 2
yhat

array([[17.4587013 ,  2.9724858 , 18.21290946,  5.91360428, 15.56538332,
        13.31881314,  4.05287638,  0.32273811,  3.75215932, 14.61206258],
       [ 4.32240779,  5.33064917,  0.76209144,  7.52115796, 12.07570169,
         8.26399902,  8.21536583,  1.33610281,  0.02038071,  1.42684448],
       [ 0.28825382, 16.34950972,  0.07133716,  4.03311009,  3.18937161,
         5.1043221 ,  1.36384376, 13.16850029,  0.02771731,  0.39320832],
       [12.20144929, 14.73635297,  1.82828837,  4.25762684, 15.4446284 ,
         0.89874838,  4.82702672,  1.08992772, 10.04904889,  2.80068946]])

In [15]:
q = np.exp(yhat)
q = q / np.expand_dims(np.sum(q, axis = 1), axis = 1)

q.sum(axis = 1)

array([1., 1., 1., 1.])

In [16]:
C_squig = np.where(p)
C_squig

(array([0, 1, 2, 3], dtype=int64), array([4, 2, 8, 6], dtype=int64))

In [19]:
H_values = -np.log(q[C_squig])
L = sum(H_values)

L

41.87928504432753

# In PyTorch:

In [20]:
L = torch.nn.CrossEntropyLoss(reduction = 'sum')
L(torch.tensor(yhat), torch.tensor(p, dtype = torch.float))

tensor(41.8793, dtype=torch.float64)